##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to Multi-Armed Bandits

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/intro_bandit"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">깃허브(GitHub) 소스 보기</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/intro_bandit.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 시작하기

Multi-Armed Bandit (MAB) is a Machine Learning framework in which an agent has to select actions (arms) in order to maximize its cumulative reward in the long term. In each round, the agent receives some information about the current state (context), then it chooses an action based on this information and the experience gathered in previous rounds. At the end of each round, the agent receives the reward associated with the chosen action.

아마도 가장 순수한 예는 MAB에 그 이름을 빌려준 문제일 것입니다. `k` 슬롯 머신(one-armed bandits)에 직면했다고 가정하고, 어떤 것이 가장 좋은 지불금을 가지고 있지만 너무 많은 돈을 잃지 않는지 알아내야 합니다.

![Multi-Armed Bandits](https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Las_Vegas_slot_machines.jpg/320px-Las_Vegas_slot_machines.jpg)

각 머신을 한 번 시도한 다음 가장 많이 지불한 머신을 선택하는 것은 좋은 전략이 아닙니다. 에이전트는 처음에는 운이 좋았지만 일반적으로 차선책인 머신을 선택하게 될 수 있습니다. 대신 에이전트는 더 많은 정보를 수집하기 위해 좋지 않은 머신을 선택하는 단계로 반복적으로 돌아와야 합니다. 이것이 Multi-Armed Bandits의 주요 과제입니다. 에이전트는 최적의 행동을 간과하지 않도록 사전 지식을 활용하는 것과 탐색 사이의 적절한 혼합을 찾아야 합니다.

MAB의 보다 실용적인 사례에는 학습자가 결정을 내릴 때마다 부수적인 정보가 포함됩니다. 이 부가 정보를 "컨텍스트"또는 "관찰"이라고 합니다.


## Multi-Armed Bandits and Reinforcement Learning

TF-Agents 라이브러리에 MAB Suite가있는 이유는 무엇입니까? RL과 MAB의 연관성은 무엇입니까? Multi-Armed Bandits는 강화 학습의 특별한 경우로 생각할 수 있습니다. [RL 소개](https://www.tensorflow.org/agents/tutorials/0_intro_rl) 를 인용하려면 :

*각 타임스텝에서 에이전트는 정책 $\pi(a_t|s_t)$에 따라 환경에 대한 행동을 취합니다. 여기서 $s_t$는 환경의 현재 관찰이며, 환경에서 보상 $r_{t+1}$과 다음 관측값 $s_{t+1}$을 받습니다. 목표는 보상(이익)의 합계를 극대화하기 위해 정책을 개선하는 것입니다.*

In the general RL case, the next observation $s_{t+1}$ depends on the previous state $s_t$ and the action $a_t$ taken by the policy. This last part is what separates MAB from RL: in MAB, the next state, which is the observation, does not depend on the action chosen by the agent.

이러한 유사성을 통해 TF-Agent에 존재하는 모든 개념을 재사용 할 수 있습니다.

- An **environment** outputs observations, and responds to actions with rewards.
- A **policy** outputs an action based on an observation, and
- An **agent** repeatedly updates the policy based on previous observation-action-reward tuples.


## The Mushroom Environment

For illustrative purposes, we use a toy example called the "Mushroom Environment". The mushroom dataset ([Schlimmer, 1981](https://archive.ics.uci.edu/ml/datasets/Mushroom)) consists of labeled examples of edible and poisonous mushrooms. Features include shapes, colors, sizes of different parts of the mushroom, as well as odor and many more.

![mushroom](https://archive.ics.uci.edu/ml/assets/MLimages/Large73.jpg)

모든 지도 학습 데이터세트와 마찬가지로, 버섯 데이터세트는 상황별 MAB 문제로 전환될 수 있습니다. [Riquelme 등. (2018)](https://arxiv.org/pdf/1802.09127.pdf)에서도 사용되는 방법을 사용합니다. 이 변환에서 에이전트는 버섯의 특징을 받아 먹거나 먹지 않기로 결정합니다. 식용 버섯을 먹으면 +5의 보상이 생기고, 독버섯을 먹으면 같은 확률로 +5 또는 -35가 됩니다. 버섯을 먹지 않으면 버섯의 종류와 관계없이 보상이 0이됩니다. 다음 표에는 보상 할당이 요약되어 있습니다.

> ```
>
> ```

```
       | edible | poisonous
```

----------- | -------- | ---------- 먹기 | +5 | -35 / +5 먹지 않기| 0 | 0

```

```

## The LinUCB Agent

Performing well in a contextual bandit environment requires a good estimate on the reward function of each action, given the observation. One possibility is to estimate the reward function with linear functions. That is, for every action $i$, we are trying to find the parameter $\theta_i\in\mathbb R^d$ for which the estimates

$r_{t, i} \sim \langle v_t, \theta_i\rangle$

are as close to the reality as possible. Here $v_t\in\mathbb R^d$ is the context received at time step $t$. Then, if the agent is very confident in its estimates, it can choose $\arg\max_{1, ..., K}\langle v_t, \theta_k\rangle$ to get the highest expected reward.

As explained above, simply choosing the arm with the best estimated reward does not lead to a good strategy. There are many different ways to mix exploitation and exploration in linear estimator agents, and one of the most famous is the Linear Upper Confidence Bound (LinUCB) algorithm (see e.g. [Li et al. 2010](https://arxiv.org/abs/1003.0146)). LinUCB has two main building blocks (with some details omitted):

1. It maintains estimates for the parameters of every arm with Linear Least Squares: $\hat\theta_i\sim X^+_i r_i$, where $X_i$ and $r_i$ are the stacked contexts and rewards of rounds where arm $i$ was chosen, and $()^+$ is the pseudo inverse.
2. It maintains *confidence ellipsoids* defined by the inverse covariance $X_i^\top X_i$ for the above estimates.

The main idea of LinUCB is that of "Optimism in the Face of Uncertainty". The agent incorporates exploration via boosting the estimates by an amount that corresponds to the variance of those estimates. That is where the confidence ellipsoids come into the picture: for every arm, the optimistic estimate is $\hat r_i = \max_{\theta\in E_i}\langle v_t, \theta\rangle$, where $E_i$ is the ellipsoid around $\hat\theta_i$. The agent chooses best looking arm $\arg\max_i\hat r_i$.

물론 위의 설명은 LinUCB가 수행하는 작업에 대한 직관적이지만 피상적인 요약입니다. 구현은 [여기](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/lin_ucb_agent.py)의 코드베이스에서 찾을 수 있습니다.

## What's Next?
If you want to have a more detailed tutorial on our Bandits library take a look at our [tutorial for Bandits](https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/bandits_tutorial.ipynb). If, instead, you would like to start exploring our library right away, you can find it [here](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits). If you are even more eager to start training, look at some of our end-to-end examples [here](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits/agents/examples/v2), including the above described mushroom environment with LinUCB [here](https://github.com/tensorflow/agents/tree/master/tf_agents/bandits/agents/examples/v2/train_eval_mushroom.py). 